# 第8章：モデル構築の準備

## 8.4 特徴量を作成する

### 8.4.3 ライブラリの宣言

In [ ]:
import numpy as np
import pandas as pd

gi_train_mm10_df = pd.read_csv("../input/gi_train_mm10.csv") # 10月のデータ
gi_train_mm11_df = pd.read_csv("../input/gi_train_mm11.csv") # 11月のデータ

# pandasのconcatで結合する
gi_train_df = pd.concat([gi_train_mm10_df, gi_train_mm11_df], axis=0)
pp_gi_train_df = gi_train_df[gi_train_df["customer_id"].notna()].copy()

# 各event_typeごとのDataFrameを作成
pp_gi_train_df_et1 = pp_gi_train_df[pp_gi_train_df["event_type"] == 1]
pp_gi_train_df_et2 = pp_gi_train_df[pp_gi_train_df["event_type"] == 2]
pp_gi_train_df_et3 = pp_gi_train_df[pp_gi_train_df["event_type"] == 3]

In [ ]:
contact_list = pp_gi_train_df_et3["customer_id"].drop_duplicates()
contact_list

In [ ]:
pp_gi_train_df_et1_rev = pp_gi_train_df_et1[pp_gi_train_df_et1["customer_id"].isin(contact_list)]
pp_gi_train_df_et1_rev.head()

In [ ]:
pp_gi_train_df_et2_rev = pp_gi_train_df_et2[pp_gi_train_df_et2["customer_id"].isin(contact_list)]
pp_gi_train_df_et2_rev.head()

### 8.4.5 属性情報とフレーム滞在時間の抽出

In [ ]:
output_df = pp_gi_train_df_et1_rev[["customer_id", "event_day", "gender", "age", "time_duration"]].copy()
output_df

### 8.4.6 カテゴリ変数のダミー変数化

In [ ]:
# カテゴリ変数をまとめたリスト
cate_cols = ["event_day", "gender", "age"]
# ダミー変数化
output_df = pd.get_dummies(data=output_df, columns=cate_cols)
output_df

### 8.4.7 各エリアの最大滞在時間の算出

In [ ]:
# エリアIDのリストを数値型で作成
area_id_list = sorted(pp_gi_train_df_et2_rev["area_id"].astype(int).unique().tolist())
area_id_list

In [ ]:
# areaのtime_duration関連のカラムを格納するリスト
area_time_duration_cols = []

# 各エリアについてループ処理
for tmp_area_id in area_id_list:
    # 当該エリアのtime_durationを取得
    #（この時点ではcustomer_idについて複数レコードが存在する）
    tmp_area_duration_df = pp_gi_train_df_et2_rev[pp_gi_train_df_et2_rev["area_id"]==tmp_area_id][["customer_id", "time_duration"]]
    # そのうちcusotmer_idごとに最大値を取得する
    tmp_area_duration_df = tmp_area_duration_df.groupby(by=["customer_id"]).max().reset_index(drop=False)

    # カラム名を指定する
    tmp_area_max_time_duration_col = f"area{tmp_area_id}_max_time_duration"
    area_time_duration_cols += [tmp_area_max_time_duration_col]
    tmp_area_duration_df = tmp_area_duration_df.rename(columns={"time_duration": tmp_area_max_time_duration_col})
    
    # customer_idをキーとしてoutput_dfに結合する
    output_df = pd.merge(output_df, tmp_area_duration_df, on=["customer_id"], how="left")

In [ ]:
output_df

In [ ]:
area_time_duration_cols

### 8.4.8 目的変数（buy_flag）の追加

In [ ]:
buy_flag_df = pp_gi_train_df_et3[pp_gi_train_df_et3["buy_flag"] == 1].drop_duplicates(subset="customer_id")[["customer_id", "buy_flag"]].copy()
buy_flag_df

In [ ]:
# マージしてoutput_dfに格納
output_df = pd.merge(output_df, buy_flag_df, on="customer_id", how="left")
# buy_flagについて0埋めとint型への変更を実施
output_df["buy_flag"] = output_df["buy_flag"].fillna(0)
output_df["buy_flag"] = output_df["buy_flag"].astype(int)
output_df

#### 8.4.9 欠損値の補間

In [ ]:
# 木系モデルの欠損値処理（-1埋め）
# エリア1～5についてループを回す
dtc_pp_gi_train_df = output_df.copy()
for col in area_time_duration_cols:
    dtc_pp_gi_train_df[col] = dtc_pp_gi_train_df[col].fillna(-1)

# ロジスティック回帰モデルの欠損値処理（0埋め）
# エリア1～5についてループを回す
lgr_pp_gi_train_df = output_df.copy()
for col in area_time_duration_cols:
    lgr_pp_gi_train_df[col] = lgr_pp_gi_train_df[col].fillna(0)

In [ ]:
dtc_pp_gi_train_df

In [ ]:
lgr_pp_gi_train_df

## 8.5 テストデータの特徴量抽出

### 8.5.1 テストデータの確認

In [ ]:
# GIのtestデータを読み込む
gi_test_df = pd.read_csv("../input/gi_test.csv")
gi_test_df.head()

### 8.5.2 テストデータの特徴量抽出

In [ ]:
# 各event_typeごとのDataFrameを作成
pp_gi_test_df_et1 = gi_test_df[gi_test_df["event_type"] == 1]
pp_gi_test_df_et2 = gi_test_df[gi_test_df["event_type"] == 2]
pp_gi_test_df_et3 = gi_test_df[gi_test_df["event_type"] == 3]

output_df = pp_gi_test_df_et1[["customer_id", "event_day", "gender", "age", "time_duration"]].copy()

# ダミー変数化
output_df = pd.get_dummies(data=output_df, columns=cate_cols)
output_df

In [ ]:
area_time_duration_cols

In [ ]:
# 各エリアについてループ処理
for tmp_area_id in area_id_list:
    # 当該エリアのtime_durationを取得
    #（この時点ではcustomer_idについて複数レコードが存在する）
    tmp_area_duration_df = pp_gi_test_df_et2[pp_gi_test_df_et2["area_id"]==tmp_area_id][["customer_id", "time_duration"]]
    # そのうちcusotmer_idごとに最大値を取得する
    tmp_area_duration_df = tmp_area_duration_df.groupby(by=["customer_id"]).max().reset_index(drop=False)

    # カラム名を指定する
    tmp_area_max_time_duration_col = f"area{tmp_area_id}_max_time_duration"
    tmp_area_duration_df = tmp_area_duration_df.rename(columns={"time_duration": tmp_area_max_time_duration_col})
    
    # customer_idをキーとしてoutput_dfに結合する
    output_df = pd.merge(output_df, tmp_area_duration_df, on=["customer_id"], how="left")

In [ ]:
output_df

In [ ]:
# 木系モデルの欠損値処理（-1埋め）
# エリア1～5についてループを回す
dtc_pp_gi_test_df = output_df.copy()
for col in area_time_duration_cols:
    dtc_pp_gi_test_df[col] = dtc_pp_gi_test_df[col].fillna(-1)

# ロジスティック回帰モデルの欠損値処理（0埋め）
# エリア1～5についてループを回す
lgr_pp_gi_test_df = output_df.copy()
for col in area_time_duration_cols:
    lgr_pp_gi_test_df[col] = lgr_pp_gi_test_df[col].fillna(0)

In [ ]:
dtc_pp_gi_test_df

In [ ]:
lgr_pp_gi_test_df

## 8.6 準備結果の出力

### 8.6.2 csvファイル形式の出力

In [ ]:
# 出力するファイルパスを設定
dtc_pp_gi_train_df_csv = "../intermediate/dtc_pp_gi_train_df_csv.csv"
dtc_pp_gi_test_df_csv = "../intermediate/dtc_pp_gi_test_df_csv.csv"
lgr_pp_gi_train_df_csv = "../intermediate/lgr_pp_gi_train_df_csv.csv"
lgr_pp_gi_test_df_csv = "../intermediate/lgr_pp_gi_test_df_csv.csv"

# csv出力
dtc_pp_gi_train_df.to_csv(dtc_pp_gi_train_df_csv, encoding="shift_jis")
dtc_pp_gi_test_df.to_csv(dtc_pp_gi_test_df_csv, encoding="shift_jis")
lgr_pp_gi_train_df.to_csv(lgr_pp_gi_train_df_csv, encoding="shift_jis")
lgr_pp_gi_test_df.to_csv(lgr_pp_gi_test_df_csv, encoding="shift_jis")

### 8.6.3 pickleファイル形式の出力

In [ ]:
# dict型変数に前処理済みデータをまとめて格納する.
pp_data_dict = {"dtc": {"train": dtc_pp_gi_train_df, "test": dtc_pp_gi_test_df}, "lgr": {"train": lgr_pp_gi_train_df, "test": lgr_pp_gi_test_df}}

In [ ]:
# 出力するpickleのファイルパスを設定
pp_data_dict_pkl_fpath = "../intermediate/pp_data_dict.pkl3"
# joblibを使用して, pickleファイルを出力する
import joblib
joblib.dump(pp_data_dict, pp_data_dict_pkl_fpath, compress=3)